In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma bs4 langchain-cohere

In [ ]:
%pip install --upgrade --quiet  nlpcloud langchain

In [ ]:
from getpass import getpass
import os

NLPCLOUD_API_KEY = getpass()
os.environ["NLPCLOUD_API_KEY"] = NLPCLOUD_API_KEY

In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_community.llms import NLPCloud
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
llm = NLPCloud()

In [ ]:
import getpass
import os

os.environ["COHERE_API_KEY"] = getpass.getpass()

In [ ]:
from langchain_cohere import CohereEmbeddings

# We gonna use Cohere Embedding model.
embeddings_model = CohereEmbeddings()

In [ ]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader("https://www.databricks.com/glossary/llmops")
docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings_model)

In [ ]:
# Retrieve and generate using the relevant snippets of the article.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# If you uses it many times, it may give you this error:
  # HTTPError: 429 Client Error: Too Many Requests for url.
  # Rate limit: maximum number of requests per hour reached.
rag_chain.invoke("What is LLMOps?")

In [ ]:
# Cleanup.
vectorstore.delete_collection()